In [41]:
import numpy as np
import scipy
import utils
from utils import HouseHolder, QR, SVD,pic_deblur
import scipy.io
import scipy.linalg
from PIL import Image
from numpy import asarray
from matplotlib import image
from matplotlib import pyplot as plt
import time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
#read the pricture
def open_picture(size:str,name:str):
    path = './test_images/'+size+'_'+size+'_'+name    
    im = Image.open(path+'.jpg')
    img = np.array(im)
    img = img.astype(np.float64) / 255
    return img


In [47]:
def working_procee(size,name,power,trunc):
    img_data = open_picture(size,name)
    #plt.imshow(img_data)
    blur_kernel_l,blur_kernel_r,blur_data = pic_deblur.blur_picture(img_data,blur_type='tridiagonal',power_l=power,power_r=power)
    begin = time.time()
    deblur_data,psnr = pic_deblur.deblur_picture(blur_kernel_l,blur_kernel_r,blur_data,trunc)
    time_used = time.time()-begin
    data = [img_data,blur_data,deblur_data]
    kernel = [blur_kernel_l,blur_kernel_r]
    return time_used,psnr,data,kernel

In [49]:
time_used,psnr,data,kernel = working_procee('1024','coffee',66,333)